### 행렬 곱
- 내적(inner product) 또는 닷 프로젝트(dot product)

### 행렬 곱 실습
- matmul()
- 행렬 간의 곱, 백터와 행렬의 곱
- 1,2 차원 행렬 계산에 용이

In [1]:
import torch
x=torch.FloatTensor([[1,2],
                     [3,4],
                     [5,6]])
y=torch.FloatTensor([[1,2],
                     [1,2]])
print(x.size(), y.size())

torch.Size([3, 2]) torch.Size([2, 2])


In [2]:
z=torch.matmul(x,y)
x.size(), x

(torch.Size([3, 2]),
 tensor([[1., 2.],
         [3., 4.],
         [5., 6.]]))

- bmm()
- 딥러닝에서는 샘플들의 연산이 행렬곱일 떄, 여러 샘플을 병렬로 연산 가능(미니 배치)
- 3차원 행렬 계산

In [3]:
x=torch.FloatTensor(3,3,2)
y=torch.FloatTensor(3,2,3)
z=torch.bmm(x,y)
z.size()

torch.Size([3, 3, 3])

### 선형 계층
- 학습: 모델을 통해 목표로 하는 함수 $f*$에 가까운 함수 $f$를 구하는 과정
- 모델: 근사 함수 $f$를 계산하는 도구
- 신경망으로 구현한 기본 모델 -> 선형 계층
- 4개의 입력을 받아 3개의 출력을 반환하는 함수 => $f: R^4$ &rarr; $R^3$
- 선형 계층: 행렬 곱셈(W)과 벡터의 덧셈(b)으로 이루어진 선형 변환
- 대량의 데이터 -> 병렬 처리 -> 미니배치
  - n차원의 특징 벡터로 구성된 샘플 N개를 모아서 행렬로 구성 &rarr; epoch

#### epoch
- 훈련 데이터셋 전체가 신경망을 한 번 완전히 통과한 과정
- ex) 데이터 셋의 크기: 1000개<br>
  $\quad$배치 크기: 100
  - 1 epoch = 10번의 파라미터 업데이트(1000개의 데이터를 100개씩 10번 나눠서 학습)
- 여러 epoch 동안 반복해서 데이터를 보게 하면서 점진적으로 오차(loss)를 줄임
- 너무 많은 epoch을 돌리면 학습 데이터에 과적합될 수 있음  

In [4]:
W=torch.FloatTensor([[1,2],[3,4],[5,6]])
b=torch.FloatTensor([2,2])

In [5]:
# 선형 계층 함수
def linear(x,W,b):
    y=torch.matmul(x,W)+b

    return y

In [6]:
x=torch.FloatTensor(4,3)
x # 무조건 0은 아님

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [7]:
y=linear(x,W,b)
y.size()

torch.Size([4, 2])

In [8]:
y

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

### torch.nn.Module 클래스 상속 받기
- torch.nn: 신경망 패키지
- Module: 신경망 구현을 위한 추상 클래스
- Module 클래스를 상속 받으려면 2개의 메서드를 오버라이드 해야함
  - \_\_init\_\_(): 계층에 필요한 변수 선언
  - forward(): 계층을 통과하는데 필요한 계산 수행(순방향 전파)

In [9]:
import torch.nn as nn

class MyLinear(nn.Module): # nn.Module을 상속받아 만든 사용자 정의 선형 계층
    def __init__(self, input_dim=3, output_dim=2): # 입력 차원, 출력 차원 정의 + 가중치/편향 초기화
        self.input_dim=input_dim # 입력 벡터 차원(특징 개수)
        self.output_dim=output_dim # 출력 벡터 차원

        super().__init__() # 부모 클래스(nn.Module)의 생성자를 호출

        self.W=torch.FloatTensor(input_dim, output_dim) # 가중치 행렬
        self.b=torch.FloatTensor(output_dim) # 편향 행렬

    def forward(self, x): # 입력 x에 대해 선형 변환 y = xW + b 수행(순전파)
        # |x|=(batch_size,input_dim)
        y=torch.matmul(x,self.W)+self.b
        return y

In [10]:
linear=MyLinear(3,2) # 변수 이름을 함수처럼 사용
y=linear(x)
y

tensor([[2.3425e+18, 2.1328e-42],
        [2.3425e+18, 2.1328e-42],
        [2.3425e+18, 2.1328e-42],
        [2.3425e+18, 2.1328e-42]])

In [11]:
y=linear.forward(x)

y

tensor([[2.3425e+18, 2.1328e-42],
        [2.3425e+18, 2.1328e-42],
        [2.3425e+18, 2.1328e-42],
        [2.3425e+18, 2.1328e-42]])

- forward() 함수를 따로 호출하지 않고 객체명에 바로 괄호를 열어 텐서 x를 인수로 넘겨줌
  - \_\_call\_\_() 함수와 forward() 함수가 매핑되어 있음(자동 호출)

### \_\_call\_\_(): 객체호출 메소드
- \_\_call\_\_(): 클래스의 인스턴스를 생성(초기화)하는 메소드
- \_\_init\_\_(): 인스턴스를 호출하는 메소드
- \_\_init\_\_()에서 forward()를 호출하는 코드를 넣어두면 인스턴스를 호출할 때마다 forward()가 자동 호출됨

### 올바른 방법: nn.Parameter 활용
- 텐서를 만들어서 nn.Parameter로 넘겨줌
  - pyTorch에서 학습할 때 전달되는 파라미터 클래스

In [12]:
class MyLinear(nn.Module):
    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim=input_dim
        self.output_dim=output_dim

        super().__init__()

        # model.parameters()에 자동 등록되어 옵티마이저가 업데이트함
        self.W=nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.b=nn.Parameter(torch.FloatTensor(output_dim))

    def forward(self, x):
        # |x| = (batch_size,input_dim)
        # |W| = (input_dim, output_dim)
        y=torch.matmul(x,self.W)+self.b # 브로드캐스팅으로 편향 추가
        # |y| = (batch_size, output_dim)
        return y

In [13]:
linear=MyLinear(3,2)
y=linear(x)
y

tensor([[2.3425e+18, 0.0000e+00],
        [2.3425e+18, 0.0000e+00],
        [2.3425e+18, 0.0000e+00],
        [2.3425e+18, 0.0000e+00]], grad_fn=<AddBackward0>)

In [14]:
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], requires_grad=True)
Parameter containing:
tensor([2.3425e+18, 0.0000e+00], requires_grad=True)


In [15]:
class MyLinear(nn.Module):
    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim=input_dim
        self.output_dim=output_dim

        super().__init__()

        self.linear=nn.Linear(input_dim, output_dim)

    def forward(self, x):
        # |x|=(batch_size,input_dim)
        y=self.linear(x)
        return y

In [16]:
linear=MyLinear(3,2)
x = torch.randn(4, 3) # batch_size = 4, input_dim = 3
y=linear(x)
y

tensor([[-0.6224,  0.0873],
        [-0.8073,  1.3279],
        [-0.2260,  0.6625],
        [ 0.7383, -1.0155]], grad_fn=<AddmmBackward0>)

### GPU 사용하기
- CUDA를 통해 GPU 연산
- 코랩 ㄱㄱ https://colab.research.google.com/?hl=ko

### 평균 제곱 오차(MSE)
- 모델에 데이터를 넣고 함수 $f$가 $f*$를 잘 근사하고 있는지 판단할 수 있어야 함
- 데이터를 넣고 원하는 출력이 나오는지 확인 <- 수치해석(numerical analysis)적 방법
- 손실(loss)값: 목표로 하는 출력(y)가 모델에 데이터를 넣어서 나오는 출력(y) 간의 차이의 크기를 모두 더한 것
  - 손실이 적을수록 모델이 함수 $f*$에 잘 근사하고 있다고 판단 => 좋은 모델
- 모델 함수는 가중치 파라미터(W, b)에 의해 동작이 정의됨
  - $\theta$로 정의: $\theta$= { W, b }
  - 모델 함수: $f_0$
- 학습의 목적: 목표 함수 $f*$에 가장 근사한 모델의 함수 $f_0$을 찾는 것 => 즉, 손실값을 최소화하는 모델을 찾는 것
  - 가장 단순한 방법: $\theta$를 랜덤하게 바꿔보는 것
- 얼마나 손실이 줄었는지 확인하는 방법: 손실함수의 결과값 확인
  - 가중치 파라미너 세타를 입력하는 손실함수

### 손실함수 선택
- L1: 벡터의 각 요소들 사이의 차이에 대핸 절댓값을 모두 더함
- L2: 유클리디안 거리(두 점 사이의 거리 계산)
- RMSE: 제곱근 평균 제곱 오차(차이가 클 때 선택하면 좋음) => 불연속
- MSE: 평균 제곱 오차(차이가 작을 때 선택하면 좋음) => 미분 가능(연속)
- CE(책에 없음): 교차 엔트로피(계산한 확률 분포와 원하는 확률 분포의 차이를 계산)
  - 손실함수의 최솟값 = CE의 최솟값

#### 엔트로피
- 확률이 높으면 정보량이 낮음
- 확률이 낮으면 정보량이 높음
- but, 정보량이 0보다는 크거나 같음(음수는 절대 아님)<br>
=> $log 1/Pi$ = $-log Pi$
- $\sum Pi$ $\log\frac{1}{pi}$ = $-Pi\sum log Pi$
- KL발산

### 손실 함수가 필요한 이유
- 신경망의 내부 가중치 파라미터를 조절해 함수를 근사계산할 수 있음
- 얼마나 잘 근사계산하는지 알아야 더 좋은 가중치 파라미터를 선택할 수 있음
- 손실 값은 얼마나 근사계산하는지 수치로 나타낸 것(낮을수록 좋음)
- 선형 계층의 가중치 파라미터 변화에 따라 손실 값이 변할 것
- 가중치 파라미터를 입력으로 받아 손실 값을 출력으로 반환하는 함수를 만들 수 있는데 이것이 손실임 

In [17]:
def mse(x_hat, x):
    y=((x-x_hat)**2).mean()

    return y

In [18]:
import torch
x=torch.FloatTensor([[1,1],[2,2]])
x_hat=torch.FloatTensor([[0,0],[0,0]])
mse(x_hat, x)

tensor(2.5000)

### torch.nn.functional 사용

In [19]:
import torch.nn.functional as f
f.mse_loss(x_hat, x)

tensor(2.5000)

In [20]:
f.mse_loss(x_hat, x, reduction='sum')

tensor(10.)

In [21]:
f.mse_loss(x_hat, x, reduction='none')

tensor([[1., 1.],
        [4., 4.]])

### torch.nn. 사용

In [22]:
import torch.nn as nn
mse_loss = nn.MSELoss()
mse_loss(x_hat, x)

tensor(2.5000)

### 교차 엔트로피

In [23]:
f.cross_entropy(x_hat, x)
CE_loss = nn.CrossEntropyLoss()
CE_loss(x_hat, x)

tensor(2.0794)